 #  Opis notatnika

Projekt na zaliczenie kursu Coders Lab. Zakłada pobranie, obrobienie i analizę danych z dedykowanego serwera za pomocą API. Dodatkowo udostępniona została dokumentacja, z którą można zapoznać się tutaj: [klik](https://api-datalab.coderslab.com/v2/docs/).

 > Dokumentacja jest czysto techniczna i ma na celu prezentację dostępnych endpointów wraz ze zwracanym typem. W celu przetestowania należy kliknąć przysisk `Authorize`, podać token (dostępny poniżej), a następnie `Try it out!` oraz uzupełnić wymagane pola (parametry requesta).

 Zgodnie z dokumentacją stwierdzamy, że udostępnione zostały nam 4 endpointy:
 - `airport` - dane o lotnisku,
 - `weather` - informacje o zarejestrowaniej pogodzie na lotnisku danego dnia,
 - `aircraft` - dane o samolotach
 - `flights` - dane o wylotach z danego lotniska per dzień.

 Wszystkie te źródła musimy pobrać, aby być w stanie wykonać całość warsztatu. W celu pobrania informacji, gdzie wymagany jest paramatr `airportId`, posłużymy się listą z pliku `airports.csv`.

 Przy wykonywaniu tego zadania możesz posłużyć się tym tokenem: `iKRsO5vdqgT903okkH1rsejOeQ0F7YC9TvutH6Wk`*.
 * Na potrzeby prezentacji token został zmieniony na niedziałający.


 #  Konfiguracja notatnika

 Tutaj zaimportuj wymagane biblioteki

In [1]:
import json
import requests
import time
import pandas as pd
from datetime import datetime, timedelta  # Biblioteka użyta do iterowania po miesiącach
from dateutil.relativedelta import relativedelta

 Tutaj zdefiniuj paramatry połączenia do API

In [2]:
# Przykładowe id, które działa: 14831

airportid = 14831
link = f'https://api-datalab.coderslab.com/api/v2/airport/{airportid}'
TOKEN = 'iKRsO5vdqgT903okkH1rsejOeQ0F7YC9TvutH6Wk'
HEADER = {'Authorization': f'{TOKEN}'}


# Sprawdzenie czy działa dla przykładowego id. MOżemy zobaczyć w jakiej postaci spodziewać się pojedynczej odpowiedzi. 
requests.get(link, headers=HEADER).json()

{'ORIGIN_AIRPORT_ID': 14831,
 'DISPLAY_AIRPORT_NAME': 'San Jose International',
 'ORIGIN_CITY_NAME': 'San Jose, CA',
 'NAME': 'SAN JOSE INTERNATIONAL AIRPORT, CA US'}

 Tutaj wczytaj plik `airports.csv` i dostosuj do dalszych kroków w celu pobierania z kolejnych endpointów. Lista lotnisk jest dostępna w kolumnie `origin_airport_id`.

In [3]:
df = pd.read_csv(
                    r'C:\\Users\\Victus\\Desktop\\Coderslab_Analityk_kurs\\Projekt_koncowy\\data\\airports.csv',
                    sep=';',  # separator uzywany do wydzielenia kolumn
                    encoding='UTF-8',
                    decimal='.'  # znak rozdzielający część całkowitą oraz ułamkową liczby
)

 #  Pobieranie `Airport`
 Zapoznaj się z dokumentacją endpointu `airport`, a następnie pobierz dane dot. poszczególnych lotnisk. Wyniki tego kroku zapisz do ramki `airport_df`, a następnie zapisz do pliku `csv`.


 Tutaj pobierz dane z endpoint'u `airport`

In [49]:
# Pobranie danych

for i in range(len(df)):
    airportid = int(df.loc[i,'origin_airport_id'])
    link = f'https://api-datalab.coderslab.com/api/v2/airport/{airportid}'
    REQ = requests.get(link, headers=HEADER).json()
    if REQ == {'error': f'Airport with id {airportid} not exist'}:  # Obsługa Airportów, których ID nie ma wśród danych
        print(REQ)
    else:
        try:
            df.loc[df['origin_airport_id'] == airportid, 'DISPLAY_AIRPORT_NAME'] = REQ['DISPLAY_AIRPORT_NAME']
        except:
            pass
        try:
            df.loc[df['origin_airport_id'] == airportid, 'ORIGIN_CITY_NAME'] = REQ['ORIGIN_CITY_NAME']
        except:
            pass
        try:
            df.loc[df['origin_airport_id'] == airportid, 'NAME'] = REQ['NAME']
        except:
            pass

{'error': 'Airport with id 10874 not exist'}


In [57]:
# Wstępne czyszczenie danych:
    # Przypisanie do airport_df df z usuniętymi NaN
    airport_df = df.dropna()

    # Zresetowanie indeksów po czyszczeniu
    airport_df = airport_df.reset_index() 

    # Usunięcie powstałej przez to kolumny ze starymi indeksami
    airport_df = airport_df.drop(['index'], axis=1) 

In [60]:
# Zapisanie do pliku

path_to_save = "C:\\Projekt_koncowy\\data\\raw\\airport_list.csv"
airport_df.to_csv(path_or_buf=path_to_save, sep=',', na_rep='', decimal='.')

 #  Pobieranie `Weather`
 Zapoznaj się z dokumentacją endpotu `Weather`, następnie pobierz dane dotyczące zarejestrowanej pogody na poszczególnych lotniskach. Wyniki zapisz do ramki `weather_df`, a później do pliku `airport_weather.csv`.


In [38]:
# Pobranie wiadomosci o pogodzie

HEADER = {'Authorization': f'{TOKEN}'}
start_date = '2019-01'
stop_date = '2020-04'


current_date = datetime.strptime(start_date, '%Y-%m')
stop_date = datetime.strptime(stop_date, '%Y-%m')
weather_list = []

# Pętla po wszystkich miesiącach

while current_date < stop_date:
    print(current_date.strftime('%Y-%m'))  # Weryfikacja która data jest obecnie obsługiwana
    link = f'https://api-datalab.coderslab.com/api/v2/airportWeather?date={current_date}'
    answer = requests.get(link, headers=HEADER).json()
    weather_list.extend(answer)     # Dopisywanie uzyskanych danych do listy
    current_date += relativedelta(months=1)
    time.sleep(1)                    # Pauza

2019-01
2019-02
2019-03
2019-04
2019-05
2019-06
2019-07
2019-08
2019-09
2019-10
2019-11
2019-12
2020-01
2020-02
2020-03


In [43]:
# Konwersja listy do data frame

airport_weather_df = pd.DataFrame(weather_list)
airport_weather_df.info()  # Sprawdzenie wymiarów uzyskanych wyników

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46226 entries, 0 to 46225
Data columns (total 33 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   WT18     0 non-null      object 
 1   STATION  46226 non-null  object 
 2   NAME     46226 non-null  object 
 3   DATE     46226 non-null  object 
 4   AWND     45845 non-null  float64
 5   PRCP     46197 non-null  float64
 6   SNOW     32338 non-null  float64
 7   SNWD     31750 non-null  float64
 8   TAVG     34625 non-null  float64
 9   TMAX     46203 non-null  float64
 10  TMIN     46200 non-null  float64
 11  WDF2     45854 non-null  float64
 12  WDF5     45704 non-null  float64
 13  WSF2     45854 non-null  float64
 14  WSF5     45704 non-null  float64
 15  WT01     16798 non-null  float64
 16  WT08     5589 non-null   float64
 17  WT02     2268 non-null   float64
 18  WT03     5085 non-null   float64
 19  WT04     362 non-null    float64
 20  WT09     316 non-null    float64
 21  WT06     522

In [45]:
# Zapisanie do pliku

path_to_save = "C:\\Coderslab_Analityk_kurs\\Projekt_koncowy\\data\\raw\\airport_weather.csv"
airport_weather_df.to_csv(path_or_buf=path_to_save, sep=',', na_rep='', decimal='.')

 #  Pobranie `Aircraft`
 Zapoznaj się z dokumentacją endpointu `aircraft`, a następnie pobierz dane produkcyjne samolotów. Wyniki zapisz do ramki `aircraft_df`, a następnie zapisz do pliku `aircraft.csv`.


In [21]:
# Pobranie wiadomosci o samolotach

link = f'https://api-datalab.coderslab.com/api/v2/aircraft'
HEADER = {'Authorization': f'{TOKEN}'}


# Zapisanie pobranych danych jako listy aircrafts

aircrafts = requests.get(link, headers=HEADER).json()


# Konwersja listy do data frame

aircraft_df = pd.DataFrame(aircrafts)

In [23]:
# Zapisanie do pliku

path_to_save = "C:\\Coderslab_Analityk_kurs\\Projekt_koncowy\\data\\raw\\aircraft.csv"
aircraft_df.to_csv(path_or_buf=path_to_save, sep=',', na_rep='', decimal='.')

 #  Pobranie `Flight`
 Zapoznaj się z dokumentacją endpointu `flights`, następnie pobierz dane dotyczące ruchu lotniczego. Wyniki zapisz do ramki `flight_df`, a później do pliku `flight.csv`.

In [6]:
# Pobranie wiadomosci o lotach

HEADER = {'Authorization': f'{TOKEN}'}
start_date = '2019-01'
stop_date = '2020-04'

# Po zdefiniowaniu początkowej i końcowej sprawdzanej daty będziemy tworzyć dane w postaci listy

current_date = datetime.strptime(start_date, '%Y-%m')
stop_date = datetime.strptime(stop_date, '%Y-%m')
flights_list = []


# Lista ID wszystkich sprawdzanych lotnisk - zapisana wcześniej:

ID_df = pd.read_csv(r'C:\\Coderslab_Analityk_kurs\\Projekt_koncowy\\data\\airports.csv',
                    sep=';',
                    encoding='UTF-8',
                    decimal='.'
                )

IDs = ID_df['origin_airport_id'].values.tolist()


for ID in IDs:
    time.sleep(1)                    # Pauza
    print("ID of current airport: ", ID)
    
    # Przyjęto założenie, że przy braku danych dla daty 2019-02 nie ma danych dla tego lotniska w ogóle
    
    link = f'https://api-datalab.coderslab.com/api/v2/flight?airportId={ID}&date=2019-02'
    REQ = requests.get(link, headers=HEADER)
    if REQ.status_code == 204:
        print("No data for this airport ")
    else:
        try:
            
            # Pętla po wszystkich miesiącach
            while current_date < stop_date:
                
                link = f'https://api-datalab.coderslab.com/api/v2/flight?airportId={ID}&date={current_date}'
                response = requests.get(link, headers=HEADER).json()
                flights_list.extend(response)     # Dopisywanie uzyskanych danych do listy
                current_date += relativedelta(months=1)
            current_date = datetime.strptime(start_date, '%Y-%m') 
        except:
            pass

ID of current airport:  10874
No data for this airport 
ID of current airport:  11233
No data for this airport 
ID of current airport:  13360
No data for this airport 
ID of current airport:  15008
No data for this airport 
ID of current airport:  11638
No data for this airport 
ID of current airport:  14150
No data for this airport 
ID of current airport:  15323
No data for this airport 
ID of current airport:  14814


KeyboardInterrupt: 

In [106]:
# Konwersja listy do data frame

flight_df = pd.DataFrame(flights_list)
flight_df.info()  # Sprawdzenie wymiarów uzyskanych wyników

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1386120 entries, 0 to 1386119
Data columns (total 27 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   MONTH                1386120 non-null  int64  
 1   DAY_OF_MONTH         1386120 non-null  int64  
 2   DAY_OF_WEEK          1386120 non-null  int64  
 3   OP_UNIQUE_CARRIER    1386120 non-null  object 
 4   TAIL_NUM             1375089 non-null  object 
 5   OP_CARRIER_FL_NUM    1386120 non-null  int64  
 6   ORIGIN_AIRPORT_ID    1386120 non-null  int64  
 7   DEST_AIRPORT_ID      1386120 non-null  int64  
 8   CRS_DEP_TIME         1386120 non-null  int64  
 9   DEP_TIME             1347712 non-null  float64
 10  DEP_DELAY_NEW        1347711 non-null  float64
 11  DEP_TIME_BLK         1386120 non-null  object 
 12  CRS_ARR_TIME         1386120 non-null  int64  
 13  ARR_TIME             1346730 non-null  float64
 14  ARR_DELAY_NEW        1344638 non-null  float64
 15

In [108]:
# Zapisanie do pliku

path_to_save = "C:\\Coderslab_Analityk_kurs\\Projekt_koncowy\\data\\raw\\flight.csv"
flight_df.to_csv(path_or_buf=path_to_save, sep=',', na_rep='', decimal='.')

 #  Podsumowanie
 W tym notatniku wykonaliśmy podstawowy krok w analizie danych - pozyskaliśmy je. Są gotowe do dalszej pracy, czyli możemy załadować je na bazę danych, a następnie zapoznać się z tym, jakie informacje ze sobą niosą. Kolejne notatniki będą służyły właśnie tym celom.